<a href="https://colab.research.google.com/github/41371109h/114-1/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A_gardio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import datetime
import gradio as gr

# --- Google Sheet 認證與連接 ---
# 第一次執行時，會跳出授權視窗，請依照指示完成認證。
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 請將這裡的網址替換成你的 Google Sheets 網址
sheet_url = 'https://docs.google.com/spreadsheets/d/1Z5JyALae6Y8ioWc4--aTTB8pjXI5f6eBif33KDIERWM/edit?gid=0#gid=0'
gsheets = gc.open_by_url(sheet_url)

# 請將這裡的 '工作表1' 替換成你的工作表名稱
worksheet = gsheets.worksheet('工作表1')

# --- 函數區：用於 Gradio 介面呼叫 ---
def get_grades_df():
    """從 Google Sheet 讀取所有成績並轉換為 DataFrame"""
    sheets_data = worksheet.get_all_values()
    df = pd.DataFrame(sheets_data[1:], columns=sheets_data[0])
    return df

def record_grade(student_name, subject, grade):
    """將單一筆成績紀錄寫入 Google Sheet"""
    # 確保成績是數字
    try:
        float(grade)
    except ValueError:
        return "⚠️ 錯誤：成績必須是數字。請重新輸入！"

    # 檢查輸入是否為空
    if not student_name or not subject or not grade:
        return "⚠️ 錯誤：姓名、科目和成績都不能為空，請重新輸入。"

    date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    new_row = [date_str, student_name, subject, grade]

    try:
        worksheet.append_row(new_row)
        # 成功寫入後，返回成功訊息和最新的 DataFrame
        return "✅ 成功新增一筆成績紀錄！", get_grades_df()
    except Exception as e:
        return f"❌ 寫入 Google Sheet 時發生錯誤：{e}", get_grades_df()

# --- Gradio 介面區 ---
with gr.Blocks(title="成績紀錄小幫手") as demo:
    gr.Markdown("# 📝 成績紀錄小幫手")
    gr.Markdown("請在下方輸入成績，並點擊『記錄成績』按鈕。")

    # 顯示現有資料的表格
    # 這裡使用 gr.Dataframe 來展示 Pandas DataFrame
    grades_table = gr.DataFrame(value=get_grades_df, label="現有成績資料")

    # 介面輸入框
    with gr.Row():
        student_name_input = gr.Textbox(label="學生姓名", placeholder="例如：王小明")
        subject_input = gr.Textbox(label="科目", placeholder="例如：數學")
        grade_input = gr.Textbox(label="成績", placeholder="例如：95")

    # 按鈕和回饋訊息
    with gr.Row():
        submit_button = gr.Button("記錄成績")
        output_message = gr.Textbox(label="訊息", interactive=False, value="")

    # 設定按鈕點擊事件
    submit_button.click(
        fn=record_grade,
        inputs=[student_name_input, subject_input, grade_input],
        outputs=[output_message, grades_table]
    )

if __name__ == "__main__":
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5920ffbde82b98b903.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
